In [ ]:
import sqlite3
import random
class Musteri:
    def __init__(self, mus_no, ad, soyad):
        self.mus_no = mus_no
        self.ad = ad
        self.soyad = soyad
    def bilgileri_goster(self):
        print(50 * "*")
        print("Musteri Bilgileri")
        print(f""" 
        Ad Soyad:{self.ad},{self.soyad}
        Musteri No:{self.mus_no}
        """)
class Hesap:
    def __init__(self, hesap_no, musteri, bakiye = 0):
        self.hesap_no = hesap_no
        self.musteri = musteri
        self.bakiye = bakiye
        self.aktif_hesap = True
    def para_yatirma(self, miktar):
        if self.aktif_hesap and miktar > 0:
            self.bakiye += miktar
            self.hesap_guncelle()
            print(f"""
            Yatirilan miktar:{miktar}
            Guncel bakiye:{self.bakiye}
            """)
        elif not self.aktif_hesap:
            print("Hesap aktif degildir.")
        else:
            print("Girilen miktar yanlistir.")
    def para_cekme(self,miktar):
        if self.aktif_hesap and miktar >0 and miktar <= self.bakiye:
            self.bakiye -= miktar
            self.hesap_guncelle()
            print(f"""
            Cekmek istediginiz tutar:{miktar}
            Guncel bakiye:{self.bakiye}
            """)
        elif not self.aktif_hesap:
            print("Hesap aktif degildir.")
        elif miktar > self.bakiye:
            print("Girilen miktar bakiyenizden fazladir.")
        else:
            print("Hatali tutar girildi.")
    def bakiye_sorgulama(self):
        if self.aktif_hesap:
            print(f"Guncel bakiyeniz {self.bakiye}")
        else:
            print("Hesap aktif degildir.")
    def hesap_kapatma(self):
        if self.aktif_hesap:
            self.aktif_hesap = False
            self.hesap_guncelle()
            print("Hesap basarili sekilde kapatildi.")
        else:
            print("Hesap aktif degildir.")
    def hesap_acma(self):
        if not self.aktif_hesap:
            self.aktif_hesap = True
            self.hesap_guncelle()
            print("Hesap aktif edildi.")
        else:
            print("Hesap zaten aktiftir.")
    def bilgileri_goster(self):
        print(50 * "*")
        print(f"""
        Hesap No:{self.hesap_no}
        Bakiye:{self.bakiye}
        """)
        self.musteri.bilgileri_goster()
    def hesap_guncelle(self):
        connection = sqlite3.connect('banka.db')
        cursor = connection.cursor()
        cursor.execute('update Hesaplar set bakiye = ?, aktif_hesap = ? where hesap_no = ?',
                    (self.bakiye, 1 if self.aktif_hesap else 0, self.hesap_no ))
        connection.commit()
        connection.close()
class Banka:
    def __init__(self):
        self.baglanti_ac()
    def baglanti_ac(self):
        self.connection = sqlite3.connect('banka.db')
        self.cursor = self.connection.cursor()
        self.cursor.execute('''create table if not exists Musteriler(
        mus_no varchar(100) primary key,
        ad varchar(100),
        soyad varchar(100)
        )''')
        self.cursor.execute('''create table if not exists Hesaplar(
        hesap_no integer primary key,
        mus_no varchar(11),
        bakiye real,
        aktif_hesap bit,
        foreign key (mus_no) references Musteriler(mus_no)
        )''')
        self.connection.commit()
    def baglanti_kapat(self):
        self.connection.close()
    def hesap_olusturma(self, musteri, ilk_bakiye = 0):
        hesap_no = random.randint(1,100)
        hesap = Hesap(hesap_no, musteri, ilk_bakiye)
        self.kaydet_musteri(musteri)
        self.kaydet_hesap(hesap)
        print(f"{hesap_no} numarali musterimiz basarili sekilde olusturuldu.")
        return hesap
    def kaydet_musteri(self,musteri):
        self.cursor.execute('insert or replace into Musteriler values(?,?,?)', (musteri.mus_no, musteri.ad,musteri.soyad))
        self.connection.commit()
    def kaydet_hesap(self,hesap):
        self.cursor.execute('insert or replace into Hesaplar values(?,?,?,?)', (hesap.hesap_no,hesap.musteri.mus_no,hesap.bakiye, 1 if hesap.aktif_hesap else 0))
        self.connection.commit()
    def tum_hesaplar(self):
        print("Tum Hesaplar:")
        self.cursor.execute('select * from Hesaplar')
        hesap_rows = self.cursor.fetchall()
        for row in hesap_rows:
            hesap_no,mus_no,bakiye,aktif_hesap = row
            musteri = self.musteri_getir(mus_no)
            hesap = Hesap(hesap_no, musteri, bakiye)
            hesap.aktif_hesap = True if aktif_hesap == 1 else False
            hesap.bilgileri_goster()
    def tum_musteriler(self):
        print("Tum Musteriler")
        self.cursor.execute('select * from Musteriler')
        musteri_rows = self.cursor.fetchall()
        for row in musteri_rows:
            mus_no, ad, soyad = row
            musteri = Musteri(mus_no, ad, soyad)
            musteri.bilgileri_goster()
    def hesap_numarasi_kontrol(self):
        hesap_no = int(input("Hesap no gir: "))
        self.cursor.execute('select * from Hesaplar where hesap_no = ?',(hesap_no,))
        hesap_row = self.cursor.fetchone()
        if hesap_row:
            musteri = self.musteri_getir(hesap_row[1])
            hesap = Hesap(hesap_row[0],hesap_row[1], hesap_row[2])
            hesap.aktif_hesap = True if hesap_row[3] == 1 else False
            return hesap
    def musteri_getir(self,mus_no):
        self.cursor.execute('select * from Musteriler where mus_no = ?',(mus_no,))
        musteri_row = self.cursor.fetchone()
        if musteri_row:
            musteri = Musteri(musteri_row[0],musteri_row[1], musteri_row[2])
            return musteri
        else:
            return None
banka = Banka()
try:
    while True:
        print(20 * "*", "Banka Yonetim Sistemi", 20 * "*")
        print(""" 
        1. Hesap Olusturma
        2. Tum Hesaplar
        3. Tum Musteriler
        4. Para Yatırma
        5. Para Cekme
        6. Bakiye Sorgulama
        7. Hesap Kapatma
        8. Hesap Acma
        9. Cikis(q-Q)
        """)
        islem = input("Yapmak istediginiz islemi girin (1-9): ")
        if islem == "1":
            ad = input("Adi girin: ")
            soyad = input("Soyad girin: ")
            mus_no = input("Musteri no girin: ")
            musteri = Musteri(mus_no,ad, soyad)
            banka.hesap_olusturma(musteri)
        elif islem == "2":
            banka.tum_hesaplar()
        elif islem == "3":
            banka.tum_musteriler()
        elif islem == "4":
            hesap = banka.hesap_numarasi_kontrol()
            if hesap:
                miktar = float(input("Yatirilacak tutari girin: "))
                hesap.para_yatirma(miktar)
        elif islem == "5":
            hesap = banka.hesap_numarasi_kontrol()
            if hesap:
                miktar = float(input("Cekmek istediginiz tutari girin: "))
                hesap.para_cekme(miktar)
        elif islem == "6":
            hesap = banka.hesap_numarasi_kontrol()
            if hesap:
                hesap.bakiye_sorgulama()
        elif islem == "7":
            hesap = banka.hesap_numarasi_kontrol()
            if hesap:
                hesap.hesap_kapatma()
        elif islem == "8":
            hesap = banka.hesap_numarasi_kontrol()
            if hesap:
                hesap.hesap_acma()
        elif islem == "9" or islem == "q" or islem == "Q":
            print("Cikis yapildi...")
            break
        else:
            print("Hatali giris yapildi.")
finally:
    banka.baglanti_kapat()

******************** Banka Yonetim Sistemi ********************
 
        1. Hesap Olusturma
        2. Tum Hesaplar
        3. Tum Musteriler
        4. Para Yatırma
        5. Para Cekme
        6. Bakiye Sorgulama
        7. Hesap Kapatma
        8. Hesap Acma
        9. Cikis(q-Q)
        
Yapmak istediginiz islemi girin (1-9): 4
Hesap no gir: 22
Yatirilacak tutari girin: 100

            Yatirilan miktar:100.0
            Guncel bakiye:100.0
            
******************** Banka Yonetim Sistemi ********************
 
        1. Hesap Olusturma
        2. Tum Hesaplar
        3. Tum Musteriler
        4. Para Yatırma
        5. Para Cekme
        6. Bakiye Sorgulama
        7. Hesap Kapatma
        8. Hesap Acma
        9. Cikis(q-Q)
        
Yapmak istediginiz islemi girin (1-9): 5
Hesap no gir: 22
Cekmek istediginiz tutari girin: 100

            Cekmek istediginiz tutar:100.0
            Guncel bakiye:0.0
            
******************** Banka Yonetim Sistemi ****************